<a href="https://colab.research.google.com/github/hoidv/CS114.P11/blob/main/create_split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [67]:
def check_valid_and_parse_filename(filename):
    #Kiểm tra đuôi file
    name, extension = os.path.splitext(filename)
    if extension not in [".jpg", ".jpeg", ".png"]:
        return None, None, None

    #Tách theo dấu "."
    parts = name.split(".")
    if len(parts) == 3:
        mssv_part, brand, order = parts[0], parts[1], parts[2]
        contributions = mssv_part.split("-")

        #Kiểm tra kiểu dữ liệu mssv, stt
        num_pattern = r"^\d+$"
        if re.match(num_pattern, order) == None:
          return None, None, None
        for contribute in contributions:
          if re.match(num_pattern, contribute) == None:
            return None, None, None

        return contributions, brand, order
    return None, None, None

In [77]:
import csv
import re

def generate_data_to_csv(data_path, save_path, csv_name):
  car_brands = {'Others': 0, 'Honda' : 1, 'Hyundai' : 2,
                    'KIA' : 3, 'Mazda' : 4, 'Mitsubishi' : 5,
                    'Suzuki' : 6, 'Toyota' : 7, 'VinFast': 8}

  #Mờ file csv để ghi data
  file_path = os.path.join(save_path, csv_name)
  with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Path", "CategoryID"])

    #Duyệt qua mỗi brand car
    for brand, brand_id in cars_brands.items():
      brand_path = os.path.join(data_path, brand)

      #Duyệt qua từng file trong thư mục brand
      for file_name in os.listdir(brand_path):
          contributions, parsed_brand, order = check_valid_and_parse_filename(file_name)

          #Kiểm tra file_name hợp lệ và brand của file_name có giống brand thư mục không
          if contributions != None:
            if parsed_brand == brand:
              writer.writerow([f"{brand}/{file_name}", brand_id])

In [75]:
import pandas as pd
import os
from sklearn.model_selection import KFold

def KFold_and_generate_csv(data, save_path, n_splits=5, random_sate=42):
  car_brands = {'Others': 0, 'Honda' : 1, 'Hyundai' : 2,
                      'KIA' : 3, 'Mazda' : 4, 'Mitsubishi' : 5,
                      'Suzuki' : 6, 'Toyota' : 7, 'VinFast': 8}

  #GroupBy data theo từng brand
  car_groups_df = car_df.groupby('CategoryID')

  #Cài đặt KFold
  kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

  #Tạo list X_train và X_val cho từng fold
  X_train = [{} for _ in range(5)]
  X_val = [{} for _ in range(5)]

  #Duyệt qua từng hãng xe tạo fold cho từng hiệu xe
  for brand_id, brand_data in car_groups_df:
    for fold_id, (train_index, val_index) in enumerate(kf.split(brand_data)):
      X_train[fold_id][brand_id] = train_index
      X_val[fold_id][brand_id] = val_index

  #Hàm lưu fold data
  def save_split_data(save_path, indices_by_brand, fold_id, dataset_type):
    #Chuyển dict df các brand thành list các df và ghép thành một df
    split_df = [brand_data.iloc[indices_by_brand[brand_id]] for brand_id, brand_data in car_groups_df]
    concate_df = pd.concat(split_df, ignore_index=True)

    #Nối file_path và tiến hành lưu file
    file_path = os.path.join(save_path, f"CarDataset-Splits-{fold_id+1}-{dataset_type}.csv")
    concate_df.to_csv(file_path, index=False)

  for fold_id in range(5):

    #Lưu file CarDataset_train
    train_indices_for_brand = X_train[fold_id]
    save_split_data(save_path, train_indices_for_brand, fold_id, dataset_type='train')

    #Lưu file CarDataset_val
    val_indices_for_brand = X_val[fold_id]
    save_split_data(save_path, val_indices_for_brand, fold_id, dataset_type='val')

In [81]:
import os

root_directory = '/content/drive/MyDrive/Colab Notebooks'
os.makedirs(root_directory, exist_ok=True)
save_path = os.path.join(root_directory, 'data')

data_path = '/content/drive/MyDrive/Public'
data_file_name = 'CarDataset.csv'
generate_data_to_csv(data_path, save_path, data_file_name)

cars_data = pd.read_csv(save_path + f'/{data_file_name}')
print(cars_data.head())
KFold_and_generate_csv(save_path, save_path)

                                      Path  CategoryID
0  Others/22521070-22520211.Others.263.jpg           0
1  Others/22521070-22520211.Others.268.jpg           0
2  Others/22521070-22520211.Others.267.jpg           0
3  Others/22521070-22520211.Others.272.jpg           0
4  Others/22521070-22520211.Others.271.jpg           0
